#  Progetto di Identificazione dei Volti con Machine Learning

## Obiettivo del Progetto
 
L'obiettivo di questo progetto è sviluppare un affidabile sistema di identificazione dei volti che, attraverso l'utilizzo di tecniche di machine learning e computer vision, sia in grado di rilevare i volti presenti in una immagine e fornire le coordinate dei bounding box che li circondano. Questo sistema dovrà operare efficientemente anche su hardware con limitate capacità di calcolo, assicurando così una vasta compatibilità e accessibilità.

**Task del Progetto**

1. **Ricerca e Selezione del Dataset**:
   - Individuare e selezionare un dataset pubblico di riconoscimento facciale adatto allo scopo.
   - Preparare i dati per l'addestramento, inclusa l'eventuale pulizia, normalizzazione e annotazione dei bounding box.

2. **Esplorazione e Preparazione dei Dati**:
   - Analizzare il dataset per comprenderne le caratteristiche e le potenziali sfide.
   - Implementare le procedure di pre-processing necessarie per preparare i dati all'addestramento.

3. **Scelta del Modello e Addestramento**:
   - Condurre una ricerca bibliografica per identificare i modelli di machine learning più adatti e meno onerosi in termini computazionali.
   - Sviluppare e addestrare il modello sul dataset selezionato, effettuando anche la validazione crociata per testarne l'efficacia.

4. **Implementazione della Pipeline di Predizione**:
   - Creare una pipeline scikit-learn che integri le fasi di pre-elaborazione dei dati, estrazione delle feature, e inferenza del modello.
   - Ottimizzare la pipeline per massimizzare le performance mantenendo bassi i requisiti computazionali.


<hr>

# 1. Il Dataset

Cominceremo a lavorare con un esempio di dataset che include sia immagini di volti sia immagini senza volti.

Il database che include i volti e le relative coordinate è stato scaricato da Kaggle, sotto il nome "Human Faces (Object Detection)", e si trova al seguente indirizzo: https://www.kaggle.com/datasets/sbaghbidi/human-faces-object-detection

Nel contesto del nostro progetto, abbiamo strutturato il dataset in una directory principale divisa in due sottodirectory: "Faces", contenente immagini con volti umani, e "No_faces", che verrà compilata con immagini casuali prive di volti. Questa organizzazione facilita l'addestramento del modello di machine learning non solo nel riconoscere volti umani ma anche nel distinguere le situazioni in cui questi non sono presenti.

All'interno della cartella "Faces", abbiamo incluso un file cruciale per il processo di addestramento, denominato "faces.csv". Questo file elenca tutte le immagini che contengono volti, specificando con precisione le coordinate dei volti all'interno di ciascuna immagine. Per le immagini raccolte nella sottodirectory "No_faces", invece, verranno annotati soltanto i nomi dei file. Quest'approccio mira a ottimizzare l'efficacia del modello nel distinguere tra immagini con e senza volti, elemento fondamentale per il successo del nostro progetto.

Iniziamo quindi ad osservare il file **faces.csv** fornito dal dataset "Human Faces (Object Detection)" :

In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("./dataset/faces.csv")
df_train

,image_name,width,height,x0,y0,x1,y1
0,00001722.jpg,1333,2000,490,320,687,664
1,00001044.jpg,2000,1333,791,119,1200,436
2,00001050.jpg,667,1000,304,155,407,331
3,00001736.jpg,626,417,147,14,519,303
4,00003121.jpg,626,418,462,60,599,166
...,...,...,...,...,...,...,...
3345,00002232.jpg,620,349,4,36,186,158
3346,00002232.jpg,620,349,122,103,344,248
3347,00002232.jpg,620,349,258,118,541,303
3348,00002232.jpg,620,349,215,11,362,108


Per il training sui casi negativi è necessario includere nel dataset un file CSV chiamato "no_faces.csv" per annotare le immagini negative. Di conseguenza, creeremo un file aggiuntivo che conterrà i nomi di tutti i file delle immagini senza volti.<br> Questo passaggio ci permetterà di addestrare il modello non solo a identificare presenze umane ma anche a riconoscere i casi negativi, migliorando la sua capacità di distinguere tra immagini con e senza volti.

In [31]:
import os

path = './dataset/no_face/'
image_name = []

for file in os.listdir(path):
    image_name.append('no_face/'+file)
    
df_out = pd.DataFrame(columns=['image_name'],data=image_name)
df_out.to_csv("./dataset/no_faces.csv",index=False)

Adesso abbiamo sia i casi positivi che i casi negativi utili per addestrare il nostro modello.

# 2. Addestramento di un classificatore a cascata

Utilizzando la documentazione online di OpenCV (https://docs.opencv.org/2.4.13.2/doc/user_guide/ug_traincascade.html) adotteremo quindi **l'addestramento di un Classificatore a Cascata.**

Seguendo la documentazione, il passo successivo, dopo aver preparato il nostro dataset, è addestrare un classificatore a cascata in OpenCV per il rilevamento dei volti. <br>
<br>Utilizzeremo **opencv_createsamples**, uno strumento specificamente progettato per generare un campione di vettori positivi. Questo processo implica combinare le immagini positive (quelle che contengono i volti che intendiamo rilevare) con le relative annotazioni. Tali annotazioni, che forniscono dettagli sui bounding box attorno ai volti presenti nelle immagini, sono tipicamente salvate in un file CSV dedicato. Per il nostro progetto, utilizzeremo il file CSV già disponibile nel dataset scaricato da Kaggle.


Tramite la funzione **opencv_createsamples**, combineremo dunque le immagini positive, ovvero quelle che includono gli oggetti di nostro interesse come i volti, con le loro annotazioni, in un unico file di vettori .vec. Questo file .vec fungerà da contenitore per tutte le informazioni relative alle immagini positive e alle loro annotazioni. È il formato richiesto dal processo di addestramento del classificatore per identificare gli oggetti di interesse nelle immagini. Questo approccio consente di semplificare e ottimizzare il processo di addestramento, fornendo al classificatore tutte le informazioni necessarie in un formato facilmente accessibile.

Nota sul file annotazioni :

E' necessario fornire manualmente un file di annotazioni che contenga le informazioni sulle posizioni dei bounding boxes degli oggetti di interesse nelle immagini positive.

Il file di annotazioni è un documento testuale che cataloga i dettagli di ogni immagine con rilevazione positiva nel formato specificato. Solitamente, ciascuna linea del file rappresenta un'immagine positiva e include le informazioni seguenti:

    1. Il percorso dell'immagine.
    2. Numero di occorrenze degli item individuati
    3. Le coordinate e le dimensioni del bounding box che circonda l'oggetto di interesse nell'immagine di riferimento.


Ecco un esempio di come potrebbe essere strutturato:


/path/to/image1.jpg w1 x1 y1 width1 height1
/path/to/image2.jpg w2 x2 y2 width2 height2
/path/to/image3.jpg w3 x3 y3 width3 height3

Dove:

  1. "/path/to/imageX.jpg" è il percorso dell'immagine con rilevazione/i positiva/e.
  2. "w" è il numero di oggetti rilevati nell'immagine
  3. "x1 y1 width1 height1, x2 y2 width2 height2, etc." costituiscono le coordinate del bounding box e le dimensioni dell'oggetto di interesse nell'immagine corrispondente.

# 3. Pre-processing per Opencv_createsamples

Per adattare il file "faces.csv" alla struttura richiesta, ogni immagine sarà documentata rispettando il formato specificato, che include:

1. Il nome dell'immagine (un record unico per ogni immagine).
2. Il numero di occorrenze (volti) presenti nell'immagine.
3. Le coordinate x, y del punto in alto a sinistra di ciascun bounding box.
4. La larghezza e l'altezza del bounding box.

Questo richiederà di modificare il file "faces.csv" affinché rifletta accuratamente queste informazioni per ogni immagine positiva, consentendo così un'efficace elaborazione e addestramento del classificatore.

In [71]:
# Preparazione iniziale del DataFrame con solo le colonne necessarie
df_train_annotations = df_train[['image_name', 'x0', 'y0', 'x1', 'y1']].copy()
df_train_annotations['image_name'] = 'faces/' + df_train_annotations['image_name']

# Calcola width e height come la differenza tra x1 e x0, e y1 e y0, rispettivamente
df_train_annotations['width'] = df_train_annotations['x1'] - df_train_annotations['x0']
df_train_annotations['height'] = df_train_annotations['y1'] - df_train_annotations['y0']

# Mi assicuro di rimuovere le righe con width o height negative
df_train_annotations = df_train_annotations[(df_train_annotations['width'] > 0) & (df_train_annotations['height'] > 0)]
df_train_annotations

,image_name,x0,y0,x1,y1,width,height
0,faces/00001722.jpg,490,320,687,664,197,344
1,faces/00001044.jpg,791,119,1200,436,409,317
2,faces/00001050.jpg,304,155,407,331,103,176
3,faces/00001736.jpg,147,14,519,303,372,289
4,faces/00003121.jpg,462,60,599,166,137,106
...,...,...,...,...,...,...,...
3345,faces/00002232.jpg,4,36,186,158,182,122
3346,faces/00002232.jpg,122,103,344,248,222,145
3347,faces/00002232.jpg,258,118,541,303,283,185
3348,faces/00002232.jpg,215,11,362,108,147,97


Adesso avendo calcolato la width e la length per ogni istanza di rettangolo identificata, procederemo ora a organizzare il file in modo che rispetti lo standard previsto da opencv_createsamples. <br>
Questo formato richiede che ciascun record corrisponda a un'unica immagine, seguito dal numero di rettangoli rilevati (bounding box) in essa, e quindi dalle quattro informazioni specifiche per ciascun rettangolo: :

1. x0
2. y0
3. Width
4. Length

Questa strutturazione consentirà di avere un file chiaro e conforme alle necessità di opencv_createsamples, facilitando l'addestramento del modello sui dati forniti.


In [72]:

def definisci_struttura(group):
    # Estrae il numero di oggetti e le loro coordinate (aggiornato per usare x0, y0, width, height)
    num_objects = group.shape[0]
    objects_str = ' '.join([' '.join(map(str, map(int, row))) for row in group[['x0', 'y0', 'width', 'height']].values])
   
    # Prende il primo nome dell'immagine nel gruppo come rappresentante per tutti
    image_name = group['image_name'].iloc[0]
    return f"{image_name} {num_objects} {objects_str}"

# Raggruppa per 'image_name' e applica la funzione di costruzione delle annotazioni secondo il formato predefinito da "Opencv_createsamples"
annotations = df_train_annotations.groupby('image_name').apply(definisci_struttura).tolist()

# Salviamo il file di annotazioni in un file
:
with open('annotations.txt', 'w') as file:
    for annotation in annotations:
        file.write(annotation + '\n')

Abbiamo tutto il necessario andiamo ora a lanciare il comando che crea il file .vec utile per l'addestramento (nb. l'esecuzione del comando prevede l'installazione di OpenCV sul sistema operativo) :

In [74]:
import subprocess

#os.chdir("./dataset/")

# Definisci il comando da eseguire
command = 'opencv_createsamples -info annotations.txt -num 1000 -w 24 -h 24 -vec positives.vec -bg no_faces.csv'

# Esegui il comando
subprocess.run(command, shell=True)


Info file name: annotations.txt
Img file name: (NULL)
Vec file name: positives.vec
BG  file name: no_faces.csv
Num: 1000
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 24
Height: 24
Max Scale: -1
RNG Seed: 12345
Create training samples from images collection...
Done. Created 1000 samples


CompletedProcess(args='opencv_createsamples -info annotations.txt -num 1000 -w 24 -h 24 -vec positives.vec -bg no_faces.csv', returncode=0)

# 4. Training del modello

Con il file .vec ora pronto, il passo successivo è procedere con l'addestramento del classificatore utilizzando opencv_traincascade. Questo strumento richiede l'utilizzo sia dei dati positivi contenuti nel file .vec che abbiamo appena preparato, sia dei dati negativi, per garantire un addestramento efficace.

Per lanciare **opencv_traincascade** eseguiremo il comando sotto riportato:

opencv_traincascade -data params -vec positives.vec -bg no_faces.csv -numPos 100 -numNeg 70

Dove:

    -data params: indica la directory "params" in questo caso, in cui verranno salvati i dati del classificatore addestrato.
    -vec positives.vec: è il file .vec che hai generato con opencv_createsamples.
    -bg bg.txt: è il file che elenca le tue immagini negative.
    -numPos 100: indica il numero di esempi positivi da utilizzare. Tipicamente, questo dovrebbe essere circa l'80-90% del numero totale di esempi positivi del file .vec.
    -numNeg 70: indica il numero di esempi negativi da utilizzare.

In [85]:
import subprocess

#os.chdir("./dataset/")

# Definisci il comando da eseguire
command = 'opencv_traincascade -data params -vec positives.vec -bg no_faces.csv -numPos 100 -numNeg 70'

# Esegui il comando
subprocess.run(command, shell=True)


PARAMETERS:
cascadeDirName: params
vecFileName: positives.vec
bgFileName: no_faces.csv
numPos: 100
numNeg: 70
numStages: 20
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : -1
stageType: BOOST
featureType: HAAR
sampleWidth: 24
sampleHeight: 24
boostType: GAB
minHitRate: 0.995
maxFalseAlarmRate: 0.5
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 100
mode: BASIC
Number of unique features given windowSize [24,24] : 162336

===== TRAINING 0-stage =====
<BEGIN
POS count : consumed   100 : 100
NEG count : acceptanceRatio    70 : 1
Precalculation time: 0
+----+---------+---------+
|  N |    HR   |    FA   |
+----+---------+---------+
|   1|        1|        1|
+----+---------+---------+
|   2|        1| 0.114286|
+----+---------+---------+
END>
Training until now has taken 0 days 0 hours 0 minutes 0 seconds.

===== TRAINING 1-stage =====
<BEGIN
POS count : consumed   100 : 100
NEG count : acceptanceRatio    70 : 0.24055
Precalculation time: 1
+----+-------

CompletedProcess(args='opencv_traincascade -data params -vec positives.vec -bg no_faces.csv -numPos 100 -numNeg 70', returncode=0)

# 5. Utilizzo del Classificatore a cascata 

Dopo aver completato con successo l'addestramento del classificatore utilizzando **opencv_traincascade** e aver ottenuto il classificatore in un file XML, possiamo ora impiegare questo classificatore per identificare gli oggetti di interesse, come i volti, nelle immagini o nei video.

Questo file XML contiene tutte le informazioni necessarie per il riconoscimento degli oggetti addestrati, permettendo al nostro software o script di OpenCV di analizzare nuove immagini o flussi video e di identificare i volti con precisione. L'utilizzo del classificatore addestrato per la rilevazione in scenari reali rappresenta l'ultima fase del processo di machine learning, consentendo l'applicazione pratica delle competenze acquisite e della tecnologia sviluppata.

In [ ]:
import cv2

# Carica il classificatore addestrato da file
cascade = cv2.CascadeClassifier('params/cascade.xml')

# Carica l'immagine su cui effettuare il rilevamento
image = cv2.imread('../dataset/faces/00000006.jpg')

# Converti l'immagine in scala di grigi (necessario per il rilevamento Haar)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Rileva gli oggetti nell'immagine
objects = cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Disegna rettangoli intorno agli oggetti rilevati
for (x, y, w, h) in objects:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Mostra l'immagine con gli oggetti rilevati
cv2.imshow('Objects detected', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# 6. Implementazione di una Pipeline 

Con tutti gli elementi a disposizione, procederemo a sviluppare una pipeline che, ricevendo in input un'immagine, sia capace di fornire in output una lista contenente le coordinate dei rettangoli che individuano i volti rilevati nell'immagine stessa.

In [ ]:
import cv2

def detect_obj(image_path,classifier_path):

    # Carica il classificatore addestrato da file
    cascade = cv2.CascadeClassifier(classifier_path)

    # Carica l'immagine su cui effettuare il rilevamento
    image = cv2.imread('image_path)

    # Converti l'immagine in scala di grigi (necessario per il rilevamento Haar)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Rileva gli oggetti nell'immagine
    objects = cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    return objects